In [1]:
# a_d = All asteroids data
# cur == cursor

In [2]:
%pip install requests
import requests
import json

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install datetime
from datetime import datetime

Note: you may need to restart the kernel to use updated packages.


In [4]:
api_key = "oyohyWG8Ci1P6n5jwLBOSLBa6yQSuLcJgEhXnd0c"

In [5]:
a_d = []         #initialising the empty list    # a_d = All asteroids data
target = 10000   # Give target as per the requirement  
url =f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-01&end_date=2024-01-08&api_key={api_key}"
# here we are using foramt strings to combine strings and number, f'' are introduced in p.v - 3.6

while len(a_d) < target:                     # while loop is used to keep making the api call and to collect the data until 10k
    response = requests.get(url)             # .get() will send the GET request to the current URL and stores it in the response variable
    data = response.json()                   # .json() converts the data stored in response into a dictionary and from here we can collect the next url
    details = data['near_earth_objects']     # here we need the NEO data of (dict_keys(['links', 'element_count', 'near_earth_objects'])) also here each key is a date and value is list
    
    for date, info in details.items():       # .items() this method returns a view object and it contains the key value pair of dictiorary as tuples in a list
        for i in info:                       # here we need to loop through every asteroid in the list
            a_d.append(dict(id = int(i['id']),
                            neo_reference_id = int(i['neo_reference_id']),
                            name = format(i['name']),
                            absolute_magnitude_h = float(i['absolute_magnitude_h']),
                            estimated_diameter_km_min = float(i['estimated_diameter']['kilometers']['estimated_diameter_min']),
                            estimated_diameter_km_max = float(i['estimated_diameter']['kilometers']['estimated_diameter_max']),
                            is_potentially_hazardous_asteroid = bool(i['is_potentially_hazardous_asteroid']),
                            close_approach_date = datetime.strptime(i['close_approach_data'][0]['close_approach_date'], '%Y-%m-%d'),
                            relative_velocity = float(i['close_approach_data'][0]['relative_velocity']['kilometers_per_hour']),
                            astronomical = float(i['close_approach_data'][0]['miss_distance']['astronomical']),
                            miss_dis_km = float(i['close_approach_data'][0]['miss_distance']['kilometers']),
                            miss_dis_lunar = float(i['close_approach_data'][0]['miss_distance']['lunar']),
                            orbiting_body = i['close_approach_data'][0]['orbiting_body']))
            if len(a_d) >= target: # here this condition will stop the loop if the target is collected
                break
        if len(a_d) >= target:
            break                  # either of the two if conditions woll break the loop if the target is collected
    start_url = data['links'].get('next')
    if not start_url:
        break         # here the pagination happens, the ['next'] collects the next page link from the response variable and stores it in the url
url = start_url             

In [6]:
a_d

[{'id': 2415949,
  'neo_reference_id': 2415949,
  'name': '415949 (2001 XY10)',
  'absolute_magnitude_h': 19.37,
  'estimated_diameter_km_min': 0.3552670883,
  'estimated_diameter_km_max': 0.7944013596,
  'is_potentially_hazardous_asteroid': False,
  'close_approach_date': datetime.datetime(2024, 1, 2, 0, 0),
  'relative_velocity': 57205.8951204341,
  'astronomical': 0.3372535274,
  'miss_dis_km': 50452409.349026635,
  'miss_dis_lunar': 131.1916221586,
  'orbiting_body': 'Earth'},
 {'id': 3160747,
  'neo_reference_id': 3160747,
  'name': '(2003 SR84)',
  'absolute_magnitude_h': 26.0,
  'estimated_diameter_km_min': 0.0167708462,
  'estimated_diameter_km_max': 0.0375007522,
  'is_potentially_hazardous_asteroid': False,
  'close_approach_date': datetime.datetime(2024, 1, 2, 0, 0),
  'relative_velocity': 38589.054833182,
  'astronomical': 0.1323425924,
  'miss_dis_km': 19798169.933318187,
  'miss_dis_lunar': 51.4812684436,
  'orbiting_body': 'Earth'},
 {'id': 3309828,
  'neo_reference_id':

In [7]:
%pip install pandas
import pandas as pd

Note: you may need to restart the kernel to use updated packages.


In [8]:
!pip install pymysql

In [9]:
import pymysql as db

In [66]:
connection = db.connect(
    host = 'localhost',
    user = 'gokul',
    password = '1234',
    database = 'neo'
)

In [67]:
connection

In [68]:
cur = connection.cursor() # cur == cursor

In [13]:
cur.execute("""create table asteroids(
id INT,
name VARCHAR(30),
absolute_magnitude_h FLOAT,
estimated_diameter_km_min FLOAT,
estimated_diameter_km_max FLOAT,
is_potentially_hazardous_asteroid BOOL
);
""")

0

In [14]:
cur.execute( """create table close_approach (
neo_reference_id INT,
close_approach_date DATE,
relative_velocity FLOAT,
astronomical FLOAT,
miss_dis_km FLOAT,
miss_dis_lunar FLOAT,
orbiting_body TEXT
);
""")

0

In [15]:
insert_values = """insert into asteroids 
(id, name, absolute_magnitude_h, estimated_diameter_km_min, estimated_diameter_km_max, is_potentially_hazardous_asteroid)
values
(%s, %s, %s, %s, %s, %s)"""
for i in a_d:
    values = (i['id'],
              i['name'],
              i['absolute_magnitude_h'],
              i['estimated_diameter_km_min'],
              i['estimated_diameter_km_max'],
              i['is_potentially_hazardous_asteroid'])
    cur.execute(insert_values, values)

connection.commit()

In [16]:
value_close_approach = """ insert into close_approach 
(neo_reference_id,close_approach_date,relative_velocity,astronomical,miss_dis_km,miss_dis_lunar,orbiting_body)
values(%s, %s, %s, %s, %s, %s, %s)"""
for i in a_d:
    values = (i['neo_reference_id'],
    i['close_approach_date'],
    i['relative_velocity'],
    i['astronomical'],
    i['miss_dis_km'],
    i['miss_dis_lunar'],
    i['orbiting_body'])
    cur.execute(value_close_approach, values)
connection.commit()

In [ ]:
# Q1 Count how many times each asteroid has approached Earth

# syntax for count is select count(column_name) from table_name;
# the count syntax allows us to count the no of rows in a tables with this we could find the no of approaches with close approach date.bb 

query_1 = """ select name, count(*) as no_of_Approaches from asteroids a
join close_approach ca 
on a.id = ca.neo_reference_id    # here Join clause joins the two table which is asteroids and close_approach
group by name                    # why we are joining the tables coz of the info we is from diff tables thats why
order by no_of_Approaches desc """        # ON clause is the condition here -> it only join the rows of ID column in the asteroids table that matches the neo_reference_id column in the close_approach table
cur.execute(query_1)                # Group by groups the results by the name
data_1 = cur.fetchall()

In [102]:
data_1


(('415949 (2001 XY10)', 5929),
 ('(2003 SR84)', 5929),
 ('(2005 YQ96)', 5929),
 ('(2009 HC21)', 5929),
 ('(2010 XA11)', 5929),
 ('(2011 YP10)', 5929),
 ('(2012 SD22)', 5929),
 ('(2017 YD8)', 5929),
 ('(2019 KK5)', 5929),
 ('613286 (2005 YQ96)', 5929),
 ('(2023 AW)', 5929),
 ('(2023 XZ4)', 5929),
 ('(2023 YR)', 5929),
 ('(2024 AA)', 5929),
 ('(2022 OY78)', 5929),
 ('(2024 AQ1)', 5929),
 ('(2024 AA2)', 5929),
 ('(2024 AR3)', 5929),
 ('(2024 BK1)', 5929),
 ('669051 (2012 SD22)', 5929),
 ('(2024 KY)', 5929),
 ('(2024 KJ1)', 5929),
 ('(2015 OD22)', 5929),
 ('(2018 BA)', 5929),
 ('(2020 BC)', 5929),
 ('(2021 SG2)', 5929),
 ('(2022 AO2)', 5929),
 ('(2023 YR1)', 5929),
 ('(2023 YY1)', 5929),
 ('(2024 AV2)', 5929),
 ('(2024 AA6)', 5929),
 ('(2025 AB)', 5929),
 ('450293 (2004 LV3)', 5929),
 ('(2010 PJ)', 5929),
 ('(2016 SU2)', 5929),
 ('(2022 BR6)', 5929),
 ('(2023 QC2)', 5929),
 ('(2023 XX12)', 5929),
 ('(2023 XS13)', 5929),
 ('(2024 AD)', 5929),
 ('(2024 AB2)', 5929),
 ('(2024 QT)', 5929),
 ('

In [ ]:
# Q2 Average velocity of each asteroid over multiple approaches

query_2 ="""select name, avg(ca.relative_velocity) as avg_velocity from asteroids a
        join close_approach ca 
        on a.id = ca.neo_reference_id
        group by name
        order by avg_velocity desc"""
cur.execute(query_2)
data_2 = cur.fetchall()

In [100]:
data_2

(('(2013 NT11)', 136267.59375),
 ('434188 (2003 AD23)', 121814.4296875),
 ('(2013 YD)', 95388.9453125),
 ('(2015 OD22)', 91415.390625),
 ('(2021 LA)', 89673.0078125),
 ('450293 (2004 LV3)', 85384.3984375),
 ('304640 (2006 WW1)', 82917.3828125),
 ('(2017 WQ1)', 77947.796875),
 ('(2024 AC3)', 77405.078125),
 ('(2021 MQ1)', 77270.3515625),
 ('(2024 AA)', 77155.59375),
 ('(2019 KK5)', 74999.265625),
 ('(2017 YA8)', 74295.3515625),
 ('(2022 BR6)', 73098.609375),
 ('(2012 DN31)', 72972.421875),
 ('(2022 AB)', 72258.140625),
 ('(2022 AO2)', 70248.9375),
 ('(2021 SG2)', 69336.4453125),
 ('(2024 BR)', 68918.578125),
 ('(2017 YD8)', 68377.1171875),
 ('(2012 BV13)', 67673.6875),
 ('(2020 AQ24)', 67654.625),
 ('(2017 PM26)', 67006.5390625),
 ('(2002 AO11)', 66352.9921875),
 ('(2010 PJ)', 66284.046875),
 ('(2012 AB11)', 66101.0078125),
 ('668499 (2012 AB11)', 66101.0078125),
 ('(2025 AK2)', 65934.8359375),
 ('(2021 QC3)', 65428.14453125),
 ('(2019 TY)', 64034.62890625),
 ('(2007 YZ)', 63326.0742187

In [ ]:
# Q3 List top 10 fastest asteroids

query_3 = """select name, max(ca.relative_velocity) as Fastest_asteroids from asteroids a
        join close_approach ca 
        on a.id = ca.neo_reference_id
        group by name
        order by Fastest_asteroids DESC
        limit 10"""          # The LIMIT clause in SQL is used to restrict the number of rows returned by a query.
cur.execute(query_3)
data_3 = cur.fetchall()

In [108]:
data_3

(('(2013 NT11)', 136268.0),
 ('434188 (2003 AD23)', 121814.0),
 ('(2013 YD)', 95388.9),
 ('(2015 OD22)', 91415.4),
 ('(2021 LA)', 89673.0),
 ('450293 (2004 LV3)', 85384.4),
 ('304640 (2006 WW1)', 82917.4),
 ('(2017 WQ1)', 77947.8),
 ('(2024 AC3)', 77405.1),
 ('(2021 MQ1)', 77270.4))

In [ ]:
# Q4 Find potentially hazardous asteroids that have approached Earth more than 3 times

query_4 ="""select name, count(*) as approach_count from asteroids a
        join close_approach ca 
        on a.id = ca.neo_reference_id
        where a.is_potentially_hazardous_asteroid = 1  # where clause filters the rows before grouping or aggregation
        group by name                                  # beacuse where claue filters induvidual rows not groups
        having approach_count > 3 """ # Having clause is used to filter groups not induvidual rows so it has to come after group clause
cur.execute(query_4)
data_4 = cur.fetchall()

In [111]:
data_4

(('(2005 YQ96)', 5929),
 ('(2012 SD22)', 5929),
 ('613286 (2005 YQ96)', 5929),
 ('669051 (2012 SD22)', 5929),
 ('450293 (2004 LV3)', 5929),
 ('(2008 WZ94)', 5929),
 ('(2017 YZ1)', 5929),
 ('585310 (2017 YZ1)', 5929),
 ('168318 (1989 DA)', 5929),
 ('(2002 AY1)', 5929),
 ('199003 (2005 WJ56)', 5929),
 ('434188 (2003 AD23)', 5929),
 ('(2025 AK2)', 5776))

In [ ]:
#Q5 Find the month with the most asteroid approaches
from datetime import datetime

query_5 = """select date_format(close_approach_date, '%Y-%m') as month, count(*) as approach_count from close_approach
        group by month
        order by approach_count desc
        limit 1"""
cur.execute(query_5)
data_5 = cur.fetchall()

In [124]:
data_5

(('2024-01', 10000),)

In [117]:
# Q6 Get the asteroid with the fastest ever approach speed

query_6 = """select name, ca.relative_velocity as Fast_app_speed from asteroids a
        join close_approach ca
        on a.id = ca.neo_reference_id
        order by Fast_app_speed DESC
        LIMIT 1"""
cur.execute(query_6)
data_6 = cur.fetchall()

In [71]:
data_6

((3645001, 136268.0),)

In [118]:
# Q7 Sort asteroids by maximum estimated diameter (descending)

query_7 = """select name, estimated_diameter_km_max from asteroids
        order by estimated_diameter_km_max desc"""
cur.execute(query_7)
data_7 = cur.fetchall()

In [119]:
data_7

(('199003 (2005 WJ56)', 1.38688),
 ('199003 (2005 WJ56)', 1.38688),
 ('199003 (2005 WJ56)', 1.38688),
 ('199003 (2005 WJ56)', 1.38688),
 ('199003 (2005 WJ56)', 1.38688),
 ('199003 (2005 WJ56)', 1.38688),
 ('199003 (2005 WJ56)', 1.38688),
 ('199003 (2005 WJ56)', 1.38688),
 ('199003 (2005 WJ56)', 1.38688),
 ('199003 (2005 WJ56)', 1.38688),
 ('199003 (2005 WJ56)', 1.38688),
 ('199003 (2005 WJ56)', 1.38688),
 ('199003 (2005 WJ56)', 1.38688),
 ('199003 (2005 WJ56)', 1.38688),
 ('199003 (2005 WJ56)', 1.38688),
 ('199003 (2005 WJ56)', 1.38688),
 ('199003 (2005 WJ56)', 1.38688),
 ('199003 (2005 WJ56)', 1.38688),
 ('199003 (2005 WJ56)', 1.38688),
 ('199003 (2005 WJ56)', 1.38688),
 ('199003 (2005 WJ56)', 1.38688),
 ('199003 (2005 WJ56)', 1.38688),
 ('199003 (2005 WJ56)', 1.38688),
 ('199003 (2005 WJ56)', 1.38688),
 ('199003 (2005 WJ56)', 1.38688),
 ('199003 (2005 WJ56)', 1.38688),
 ('199003 (2005 WJ56)', 1.38688),
 ('199003 (2005 WJ56)', 1.38688),
 ('199003 (2005 WJ56)', 1.38688),
 ('199003 (200

In [125]:
# Q8 Asteroids whose closest approach is getting nearer over time(Hint: Use ORDER BY close_approach_date and look at miss_distance)

query_8 = """ select name, date_format(ca.close_approach_date, '%Y-%m'), ca.miss_dis_km as miss_distance_km from asteroids a
        join close_approach ca 
        on a.id = ca.neo_reference_id
        order by name, ca.close_approach_date """
cur.execute(query_8)
data_8 = cur.fetchall()


In [126]:
data_8

(('(2002 AO11)', '2024-01', 60009500.0),
 ('(2002 AO11)', '2024-01', 60009500.0),
 ('(2002 AO11)', '2024-01', 60009500.0),
 ('(2002 AO11)', '2024-01', 60009500.0),
 ('(2002 AO11)', '2024-01', 60009500.0),
 ('(2002 AO11)', '2024-01', 60009500.0),
 ('(2002 AO11)', '2024-01', 60009500.0),
 ('(2002 AO11)', '2024-01', 60009500.0),
 ('(2002 AO11)', '2024-01', 60009500.0),
 ('(2002 AO11)', '2024-01', 60009500.0),
 ('(2002 AO11)', '2024-01', 60009500.0),
 ('(2002 AO11)', '2024-01', 60009500.0),
 ('(2002 AO11)', '2024-01', 60009500.0),
 ('(2002 AO11)', '2024-01', 60009500.0),
 ('(2002 AO11)', '2024-01', 60009500.0),
 ('(2002 AO11)', '2024-01', 60009500.0),
 ('(2002 AO11)', '2024-01', 60009500.0),
 ('(2002 AO11)', '2024-01', 60009500.0),
 ('(2002 AO11)', '2024-01', 60009500.0),
 ('(2002 AO11)', '2024-01', 60009500.0),
 ('(2002 AO11)', '2024-01', 60009500.0),
 ('(2002 AO11)', '2024-01', 60009500.0),
 ('(2002 AO11)', '2024-01', 60009500.0),
 ('(2002 AO11)', '2024-01', 60009500.0),
 ('(2002 AO11)',

In [132]:
# Q9 Display the name of each asteroid along with the date and miss distance of its closest approach to Earth

query_10 = """select name, date_format(ca.close_approach_date, '%Y-%m'), ca.miss_dis_km from asteroids a
          join close_approach ca 
          on a.id = ca.neo_reference_id
          where (a.id, ca.miss_dis_km)
          in( select neo_reference_id, min(miss_dis_km) from close_approach
          group by neo_reference_id )"""
cur.execute(query_10)
data_10 = cur.fetchall()
       

In [131]:
data_10

(('415949 (2001 XY10)', datetime.date(2024, 1, 2), 50452400.0),
 ('415949 (2001 XY10)', datetime.date(2024, 1, 2), 50452400.0),
 ('415949 (2001 XY10)', datetime.date(2024, 1, 2), 50452400.0),
 ('415949 (2001 XY10)', datetime.date(2024, 1, 2), 50452400.0),
 ('415949 (2001 XY10)', datetime.date(2024, 1, 2), 50452400.0),
 ('415949 (2001 XY10)', datetime.date(2024, 1, 2), 50452400.0),
 ('415949 (2001 XY10)', datetime.date(2024, 1, 2), 50452400.0),
 ('415949 (2001 XY10)', datetime.date(2024, 1, 2), 50452400.0),
 ('415949 (2001 XY10)', datetime.date(2024, 1, 2), 50452400.0),
 ('415949 (2001 XY10)', datetime.date(2024, 1, 2), 50452400.0),
 ('415949 (2001 XY10)', datetime.date(2024, 1, 2), 50452400.0),
 ('415949 (2001 XY10)', datetime.date(2024, 1, 2), 50452400.0),
 ('415949 (2001 XY10)', datetime.date(2024, 1, 2), 50452400.0),
 ('415949 (2001 XY10)', datetime.date(2024, 1, 2), 50452400.0),
 ('415949 (2001 XY10)', datetime.date(2024, 1, 2), 50452400.0),
 ('415949 (2001 XY10)', datetime.date(20

In [ ]:
# Q 10 List names of asteroids that approached Earth with velocity > 50,000 km/h

query_10 = """ select distinct a.name from asteroids a
        join close_approach ca 
        on a.id = ca.neo_reference_id
        where ca.relative_velocity > 50000 """
cur.execute(query_10)
data_10 = cur.fetchall()

In [155]:
data_10

(('415949 (2001 XY10)',),
 ('(2005 YQ96)',),
 ('(2017 YD8)',),
 ('(2019 KK5)',),
 ('613286 (2005 YQ96)',),
 ('(2024 AA)',),
 ('(2024 AQ1)',),
 ('(2024 AA2)',),
 ('(2015 OD22)',),
 ('(2018 BA)',),
 ('(2021 SG2)',),
 ('(2022 AO2)',),
 ('450293 (2004 LV3)',),
 ('(2010 PJ)',),
 ('(2016 SU2)',),
 ('(2022 BR6)',),
 ('(2008 WZ94)',),
 ('(2012 AB11)',),
 ('(2012 LJ2)',),
 ('(2013 NT11)',),
 ('(2017 WQ1)',),
 ('(2023 XN7)',),
 ('668499 (2012 AB11)',),
 ('304640 (2006 WW1)',),
 ('(2002 AO11)',),
 ('(2017 PM26)',),
 ('(2021 MQ1)',),
 ('(2023 YX1)',),
 ('(2024 BR)',),
 ('(2013 YD)',),
 ('(2016 RW19)',),
 ('(2017 YA8)',),
 ('(2021 QC3)',),
 ('(2023 KF5)',),
 ('(2023 OG5)',),
 ('(2023 WX)',),
 ('(2002 AY1)',),
 ('(2012 DN31)',),
 ('(2019 TY)',),
 ('(2022 AB)',),
 ('(2020 AQ24)',),
 ('(2022 YD6)',),
 ('(2024 AC3)',),
 ('199003 (2005 WJ56)',),
 ('434188 (2003 AD23)',),
 ('(2007 YZ)',),
 ('(2012 BV13)',),
 ('(2019 PJ)',),
 ('(2021 LA)',),
 ('(2025 AK2)',))

In [ ]:
# 11 Count how many approaches happened per month

query_11 = """ select date_format(close_approach_date, '%Y-%m') as month, COUNT(*) as approach_count from close_approach
        group by month
        order by month """
cur.execute(query_11)
data_11 = cur.fetchall()

In [154]:
data_11

(('2024-01', 10000),)

In [ ]:
# Q 12 Find asteroid with the highest brightness (lowest magnitude value)

query_12 = """select name, absolute_magnitude_h from asteroids
        order by absolute_magnitude_h asc
        limit 1 """
cur.execute(query_12)
data_12 = cur.fetchall()

In [153]:
data_12

(('199003 (2005 WJ56)', 18.16),)

In [ ]:
# Q 13 Get number of hazardous vs non-hazardous asteroids

query_13 = """ select is_potentially_hazardous_asteroid, count(*) as count from asteroids
           group by is_potentially_hazardous_asteroid """
cur.execute(query_13)
data_10 = cur.fetchall()

In [156]:
# Q 14 Find asteroids that passed closer than the Moon (lesser than 1 LD), along with their close approach date and distance.

query_14 = """ select a.name,  date_format(ca.close_approach_date, '%Y-%m'), ca.miss_dis_lunar from asteroids a
        join close_approach ca 
        on a.id = ca.neo_reference_id
        where ca.miss_dis_lunar < 1 """
cur.execute(query_14)
data_14 = cur.fetchall()

In [157]:
data_14

(('(2024 AD)', '2024-01', 0.631031),
 ('(2024 AD)', '2024-01', 0.631031),
 ('(2024 AD)', '2024-01', 0.631031),
 ('(2024 AD)', '2024-01', 0.631031),
 ('(2024 AD)', '2024-01', 0.631031),
 ('(2024 AD)', '2024-01', 0.631031),
 ('(2024 AD)', '2024-01', 0.631031),
 ('(2024 AD)', '2024-01', 0.631031),
 ('(2024 AD)', '2024-01', 0.631031),
 ('(2024 AD)', '2024-01', 0.631031),
 ('(2024 AD)', '2024-01', 0.631031),
 ('(2024 AD)', '2024-01', 0.631031),
 ('(2024 AD)', '2024-01', 0.631031),
 ('(2024 AD)', '2024-01', 0.631031),
 ('(2024 AD)', '2024-01', 0.631031),
 ('(2024 AD)', '2024-01', 0.631031),
 ('(2024 AD)', '2024-01', 0.631031),
 ('(2024 AD)', '2024-01', 0.631031),
 ('(2024 AD)', '2024-01', 0.631031),
 ('(2024 AD)', '2024-01', 0.631031),
 ('(2024 AD)', '2024-01', 0.631031),
 ('(2024 AD)', '2024-01', 0.631031),
 ('(2024 AD)', '2024-01', 0.631031),
 ('(2024 AD)', '2024-01', 0.631031),
 ('(2024 AD)', '2024-01', 0.631031),
 ('(2024 AD)', '2024-01', 0.631031),
 ('(2024 AD)', '2024-01', 0.631031),
 

In [158]:
# Q 15 Find asteroids that came within 0.05 AU(astronomical distance)

query_15 = """ select name, date_format(ca.close_approach_date, '%Y-%m'), ca.astronomical from asteroids a
        join close_approach ca 
        on a.id = ca.neo_reference_id
        where ca.astronomical < 0.05 """
cur.execute(query_15)
data_15 = cur.fetchall()

In [159]:
data_15

(('(2019 KK5)', '2024-01', 0.0260973),
 ('(2019 KK5)', '2024-01', 0.0260973),
 ('(2019 KK5)', '2024-01', 0.0260973),
 ('(2019 KK5)', '2024-01', 0.0260973),
 ('(2019 KK5)', '2024-01', 0.0260973),
 ('(2019 KK5)', '2024-01', 0.0260973),
 ('(2019 KK5)', '2024-01', 0.0260973),
 ('(2019 KK5)', '2024-01', 0.0260973),
 ('(2019 KK5)', '2024-01', 0.0260973),
 ('(2019 KK5)', '2024-01', 0.0260973),
 ('(2019 KK5)', '2024-01', 0.0260973),
 ('(2019 KK5)', '2024-01', 0.0260973),
 ('(2019 KK5)', '2024-01', 0.0260973),
 ('(2019 KK5)', '2024-01', 0.0260973),
 ('(2019 KK5)', '2024-01', 0.0260973),
 ('(2019 KK5)', '2024-01', 0.0260973),
 ('(2019 KK5)', '2024-01', 0.0260973),
 ('(2019 KK5)', '2024-01', 0.0260973),
 ('(2019 KK5)', '2024-01', 0.0260973),
 ('(2019 KK5)', '2024-01', 0.0260973),
 ('(2019 KK5)', '2024-01', 0.0260973),
 ('(2019 KK5)', '2024-01', 0.0260973),
 ('(2019 KK5)', '2024-01', 0.0260973),
 ('(2019 KK5)', '2024-01', 0.0260973),
 ('(2019 KK5)', '2024-01', 0.0260973),
 ('(2019 KK5)', '2024-01'

In [163]:
# Q16 Average estimated diameter of hazardous vs. non-hazardous asteroids

query_16 = """ select is_potentially_hazardous_asteroid,  avg(estimated_diameter_km_min) as avg_diameter_min, avg(estimated_diameter_km_max) as avg_diameter_max from asteroids
group by is_potentially_hazardous_asteroid """
cur.execute(query_16)
data_16 = cur.fetchall()

In [161]:
data_16

((0, 0.06316808707221773, 0.14124813674700756),
 (1, 0.2929841587096453, 0.6551324963569641))

In [166]:
# Q 17 Top 5 brightest hazardous asteroids (lowest magnitude)

query_17 = """ select name, absolute_magnitude_h from asteroids
where is_potentially_hazardous_asteroid = 1
order by absolute_magnitude_h ASC
limit 5 """
cur.execute(query_17)
data_17 = cur.fetchall()

In [167]:
data_17

(('199003 (2005 WJ56)', 18.16),
 ('199003 (2005 WJ56)', 18.16),
 ('199003 (2005 WJ56)', 18.16),
 ('199003 (2005 WJ56)', 18.16),
 ('199003 (2005 WJ56)', 18.16))

In [169]:
# Q 18 Asteroids with the greatest difference between estimated min and max diameter

query_18 = """ select name, (estimated_diameter_km_max - estimated_diameter_km_min) as diameter_dif from asteroids
order by diameter_dif desc
limit 10 """
cur.execute(query_18)
data_18 = cur.fetchall()

In [170]:
data_18

(('199003 (2005 WJ56)', 0.7666503190994263),
 ('199003 (2005 WJ56)', 0.7666503190994263),
 ('199003 (2005 WJ56)', 0.7666503190994263),
 ('199003 (2005 WJ56)', 0.7666503190994263),
 ('199003 (2005 WJ56)', 0.7666503190994263),
 ('199003 (2005 WJ56)', 0.7666503190994263),
 ('199003 (2005 WJ56)', 0.7666503190994263),
 ('199003 (2005 WJ56)', 0.7666503190994263),
 ('199003 (2005 WJ56)', 0.7666503190994263),
 ('199003 (2005 WJ56)', 0.7666503190994263))

In [171]:
# Q 19  Top 10 asteroids with the most close approaches in a single month

query_19 = """ select a.name, date_format(ca.close_approach_date, '%Y-%m') as month, count(*) as approaches from asteroids a
join close_approach ca 
on a.id = ca.neo_reference_id
group by a.name, month
order by approaches DESC
limit 10 """
cur.execute(query_19)
data_19 = cur.fetchall()

In [172]:
data_19

(('415949 (2001 XY10)', '2024-01', 5929),
 ('(2003 SR84)', '2024-01', 5929),
 ('(2005 YQ96)', '2024-01', 5929),
 ('(2009 HC21)', '2024-01', 5929),
 ('(2010 XA11)', '2024-01', 5929),
 ('(2011 YP10)', '2024-01', 5929),
 ('(2012 SD22)', '2024-01', 5929),
 ('(2017 YD8)', '2024-01', 5929),
 ('(2019 KK5)', '2024-01', 5929),
 ('613286 (2005 YQ96)', '2024-01', 5929))

In [174]:
# Q20 Average miss distance per asteroid

query_20 = """ select a.name, avg(ca.miss_dis_km ) as avg_miss_distance_km from asteroids a
join close_approach ca 
on a.id = ca.neo_reference_id
group by a.name
order by avg_miss_distance_km asc """
cur.execute(query_20)
data_20 = cur.fetchall()

In [175]:
data_20

(('(2024 AD)', 242675.9375),
 ('(2024 AA)', 610571.8125),
 ('(2024 AB2)', 744109.8125),
 ('(2024 AM)', 1209360.125),
 ('(2024 AV2)', 1452207.875),
 ('(2024 AL2)', 1496555.75),
 ('(2024 AC3)', 1562543.125),
 ('(2023 YR)', 1729961.75),
 ('(2024 AX3)', 1844629.125),
 ('(2024 AC)', 2314264.0),
 ('(2023 YR1)', 2484569.25),
 ('(2024 AM3)', 2680032.25),
 ('(2024 AH3)', 2692217.75),
 ('(2024 AH1)', 3441304.75),
 ('(2019 PJ)', 3719706.25),
 ('(2023 YX1)', 3877187.75),
 ('(2019 KK5)', 3904096.75),
 ('(2023 YY1)', 4058907.0),
 ('(2024 AW)', 4280156.5),
 ('(2024 AV3)', 4674438.0),
 ('(2024 AD2)', 4831184.5),
 ('(2024 AA3)', 5207160.0),
 ('(2024 BM1)', 5465304.5),
 ('(2002 AY1)', 5803016.5),
 ('(2024 AQ1)', 6556678.5),
 ('(2024 AA6)', 7351471.5),
 ('(2024 AR3)', 7878470.5),
 ('(2023 YV1)', 8449916.0),
 ('(2023 XX12)', 8499676.0),
 ('(2024 AS)', 9293487.0),
 ('(2023 UO9)', 10059757.0),
 ('(2011 YP10)', 10545147.0),
 ('(2018 WA2)', 12430133.0),
 ('(2024 KJ1)', 16030952.0),
 ('(2023 AW)', 16252313.0),